In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from numpy import expand_dims
from sklearn.model_selection import StratifiedKFold
from keras import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(16,(3,3), activation='relu', input_shape=(28,28,1),padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(32,(5,5),activation='relu',padding='same')) 
model.add(BatchNormalization())
model.add(Conv2D(32,(5,5),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(32,(5,5),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((3,3)))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64,(5,5),activation='relu',padding='same')) 
model.add(BatchNormalization())
model.add(MaxPooling2D((3,3)))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from numpy import expand_dims
from sklearn.model_selection import StratifiedKFold
from keras import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam

train = pd.read_csv('/kaggle/input/mniststt/train.csv')
test = pd.read_csv('/kaggle/input/mniststt/test.csv')
sub = pd.read_csv('/kaggle/input/mniststt/submission.csv')

# check the distribution of 'digit'
train['digit'].value_counts()

# drop columns
train2 = train.drop(['id','digit','letter'],1)
test2 = test.drop(['id','letter'],1)

# convert pandas dataframe to numpy array
train2 = train2.values
test2 = test2.values

plt.imshow(train2[100].reshape(28,28))

train2 = train2.reshape(-1,28,28,1)
test2 = test2.reshape(-1,28,28,1)

# normalization
train2 = train2/255.
test2 = test2/255.

# image data generator * data augmentation
idg = ImageDataGenerator(height_shift_range = (-1,1), width_shift_range = (-1,1))
idg2 = ImageDataGenerator()

# show augmented image data
sample_data = train2[100].copy()
sample = expand_dims(sample_data,0)
sample_datagen = ImageDataGenerator(height_shift_range=(-3,3), width_shift_range=(3,-3))
sample_generator = sample_datagen.flow(sample, batch_size=1)

plt.figure(figsize=(16,10))

for i in range(9):
    plt.subplot(3,3,i+1)
    sample_batch = sample_generator.next()
    sample_image = sample_batch[0]
    plt.imshow(sample_image.reshape(28,28))

# cross validation
skf = StratifiedKFold(n_splits=40, random_state=42, shuffle=True)

# patience 만큼의 epoch에도 벗어나지 못하면 callback 호출
# verbose
# factor: 학습률을 1/2로 줄인다 patience 만큼의 epoch에도 벗어나지 못하면 callback 호출
reLR = ReduceLROnPlateau(monitor='accuracy',patience=100, verbose=1, factor=0.5)
es = EarlyStopping(patience=160, verbose=1)

val_loss_min = []
result=0
nth = 0



In [ ]:
from tqdm import tqdm
for train_idx, valid_idx in tqdm(skf.split(train2,train['digit'])):
    mc = ModelCheckpoint('best_cvision.h5', save_best_only=True, verbose=1)

    x_train = train2[train_idx]
    x_valid = train2[valid_idx]
    y_train = train['digit'][train_idx]
    y_valid = train['digit'][valid_idx]

    train_generator = idg.flow(x_train, y_train, batch_size=8)
    valid_generator = idg2.flow(x_valid, y_valid)
    test_generator = idg2.flow(test2,shuffle=False)

    model = Sequential()
    model.add(Conv2D(16,(3,3), activation='relu', input_shape=(28,28,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(5,5),activation='relu',padding='same')) 
    model.add(BatchNormalization())
    model.add(Conv2D(32,(5,5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(5,5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3,3)))
    model.add(Dropout(0.3))

    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(5,5),activation='relu',padding='same')) 
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3,3)))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(128,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(10, activation='softmax'))

    # model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.002,epsilon=None), metrics=['acc'])
    model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.002,epsilon=None), metrics=['acc'])

    # learning_history = model.fit_generator(train_generator, 
    #                                        epochs=2000, 
    #                                        validation_data=valid_generator,
    #                                        callbacks=[es,mc,reLR])

    learning_history = model.fit(train_generator, 
                                         epochs=2000, 
                                         validation_data=valid_generator,
                                         callbacks=[es,mc,reLR])

    # predict
    model.load_weights('best_cvision.h5')
    result += model.predict_generator(test_generator,verbose=True)

    # save val_loss
    hist = pd.DataFrame(learning_history.history)
    val_loss_min.append(hist['val_loss'].min())

    nth += 1
    print(nth, '번째 학습을 완료했습니다.')

display(val_loss_min, np.mean(val_loss_min))model.summary()

sub.to_csv('result.csv', index=False)

In [ ]:
sub['digit'] = result.argmax(1)

In [ ]:
sub.to_csv('Dacon_cvision_0914_40_epsNone.csv',index=False)